1.build a Streamlit web app in Colab that:

Uploads PDF, TXT, DOCX files

Extracts text

Uses Gemini (via API) to generate:

MCQs, MSQs, Fill in the Blank questions

Includes answers + explanation

Allows download of results (TXT/PDF)

 2. Set Up in Google Colab

In [ ]:
# Install dependencies
!pip install streamlit fpdf pdfplumber python-docx google-generativeai pyngrok


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=2d67b5ed8ff532283f284e91cbd0b26935b7c3204e433cc4178248b89a0a5279
  Stored in directory: /

3. Create app.py file in Colab

In [ ]:
%%writefile app.py
import os
import pdfplumber
import docx
from fpdf import FPDF
import google.generativeai as genai
import streamlit as st

# Set your Gemini API key here
os.environ["GOOGLE_API_KEY"] = "your google api key"  # 🔁 Replace with your actual key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("models/gemini-1.5-pro")

# Extract text from uploaded file
def extract_text(file):
    ext = file.name.split('.')[-1].lower()
    if ext == 'pdf':
        with pdfplumber.open(file) as pdf:
            return ''.join([page.extract_text() or '' for page in pdf.pages])
    elif ext == 'docx':
        doc = docx.Document(file)
        return ' '.join([para.text for para in doc.paragraphs])
    elif ext == 'txt':
        return file.read().decode('utf-8')
    return ""

# Generate Questions with difficulty
def generate_questions(text, num_q, difficulty):
    prompt = f"""
You are an AI tutor generating {difficulty.lower()} level educational questions from this content:
\"\"\"{text}\"\"\"

Generate {num_q} questions of {difficulty.lower()} difficulty, with a mix of:
- MCQ (one correct)
- MSQ (multiple correct)
- Fill in the Blank
- Math problems with step-by-step solutions
- Diagram descriptions if needed

Each should follow this format:

### QUESTION
Type: [MCQ / MSQ / Fill in the Blank / Math]
Question: [state the full question, include diagram if needed]
A) [option A] (skip for math or fill-in-the-blank)
B) [option B]
C) [option C]
D) [option D]
Correct Answer: [e.g., A / A, C / full answer]
Explanation: [explanation or step-by-step math solution]

Now generate the questions:
"""
    response = model.generate_content(prompt)
    return response.text.strip()

# Save as PDF
def save_as_pdf(content, filename="questions.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    for block in content.split("### QUESTION"):
        if block.strip():
            pdf.multi_cell(0, 10, f"### QUESTION{block.strip()}")
            pdf.ln(5)
    pdf.output(filename)
    return filename

# Streamlit app
st.title("📘 AI Question Generator")

uploaded_file = st.file_uploader("Upload a file (PDF, DOCX, TXT)", type=['pdf', 'docx', 'txt'])
num_q = st.number_input("Number of questions", min_value=1, max_value=50, value=5)
difficulty = st.selectbox("Select difficulty level", ["Easy", "Intermediate", "Tough"])

if uploaded_file:
    text = extract_text(uploaded_file)
    if st.button("Generate Questions"):
        with st.spinner("Generating questions..."):
            questions = generate_questions(text, num_q, difficulty)
            st.success("✅ Questions generated!")
            st.text_area("Preview", questions, height=400)

            # Save files
            with open("questions.txt", "w", encoding='utf-8') as f:
                f.write(questions)
            pdf_file = save_as_pdf(questions)

            st.download_button("📄 Download TXT", data=open("questions.txt", "rb"), file_name="questions.txt")
            st.download_button("📘 Download PDF", data=open(pdf_file, "rb"), file_name="questions.pdf")


Overwriting app.py


4. Run the Streamlit App in Colab

In [ ]:
from pyngrok import ngrok

# Kill old tunnels
!pkill streamlit
!ngrok authtoken YOUR_AUTHTOKEN# Replace <YOUR_AUTHTOKEN> with your actual authtoken

# Run the app
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

!streamlit run app.py &>/content/log.txt &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://c1dd-34-145-192-155.ngrok-free.app" -> "http://localhost:8501"
